# ApacheSpark - ML - Preprocesamiento

<p><strong>Objetivo: </strong> El objetivo de este cuaderno es aprender sentencias pyspark para el preprocesamiento de los datos:</p>

## Identificación y tratamiento de valores faltantes

In [0]:
df = spark.createDataFrame(
[
('Store 1',1,448),
('Store 1',2,None),
('Store 1',3,499),
('Store 1',44,432),
(None,None,None),
('Store 2',1,355),
('Store 2',1,355),
('Store 2',None,345),
('Store 2',3,387),
('Store 2',4,312),
],
['Store','WeekInMonth','Revenue']
)

Indentificación

In [0]:
display(df.filter(df.Revenue.isNull()))

Store,WeekInMonth,Revenue
Store 1,2,null
null,null,null


In [0]:
from pyspark.sql.functions import count, when, isnull
display(df.select(
[count(when(isnull(c), c)).alias(c) for c in df.columns]
))

Store,WeekInMonth,Revenue
1,2,2


Eliminado registros con valores faltantes

In [0]:
df2 = df.dropna()
display(df2)

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,3,499
Store 1,44,432
Store 2,1,355
Store 2,1,355
Store 2,3,387
Store 2,4,312


In [0]:
df2 = df.dropna('all')
display(df2)

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,null
Store 1,3,499
Store 1,44,432
Store 2,1,355
Store 2,1,355
Store 2,null,345
Store 2,3,387
Store 2,4,312


Hay una cosa importante a tener en cuenta sobre fillna: solo realizará la operación de intercambio para tipos de columna coincidentes. Entonces, si usa un valor numérico para una columna de cadena o al revés, no funcionará.

Sustituyendo por un valor:

In [0]:
display(df.fillna(0))
display(df.fillna(0, ['Revenue']))
display(df.fillna({'WeekInMonth' : 2, 'Revenue' : 3}))

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,0
Store 1,3,499
Store 1,44,432
null,0,0
Store 2,1,355
Store 2,1,355
Store 2,0,345
Store 2,3,387
Store 2,4,312


Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,0
Store 1,3,499
Store 1,44,432
null,null,0
Store 2,1,355
Store 2,1,355
Store 2,null,345
Store 2,3,387
Store 2,4,312


Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,3
Store 1,3,499
Store 1,44,432
null,2,3
Store 2,1,355
Store 2,1,355
Store 2,2,345
Store 2,3,387
Store 2,4,312


Sustituyendo con la media

In [0]:
from pyspark.sql.functions import mean
df.select(mean(df.Revenue)).show()

+------------+
|avg(Revenue)|
+------------+
|     391.625|
+------------+



In [0]:
display(df.fillna(391.625, ['Revenue']))

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,391
Store 1,3,499
Store 1,44,432
null,null,391
Store 2,1,355
Store 2,1,355
Store 2,null,345
Store 2,3,387
Store 2,4,312


## Eliminando duplicados

In [0]:
display(df.dropDuplicates())

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,null
Store 1,3,499
null,null,null
Store 1,44,432
Store 2,1,355
Store 2,null,345
Store 2,4,312
Store 2,3,387


In [0]:
display(df.dropDuplicates(['Store','WeekInMonth']))

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,null
Store 1,3,499
null,null,null
Store 1,44,432
Store 2,1,355
Store 2,null,345
Store 2,3,387
Store 2,4,312


## Eliminando columnas

In [0]:
display(df.drop('Revenue'))
display(df.drop('Revenue','Store'))

Store,WeekInMonth
Store 1,1
Store 1,2
Store 1,3
Store 1,44
null,null
Store 2,1
Store 2,1
Store 2,null
Store 2,3
Store 2,4


WeekInMonth
1
2
3
44
null
1
1
null
3
4


## Identificando y resolviendo valores inconsistentes

In [0]:
display(df)

Store,WeekInMonth,Revenue
Store 1,1,448
Store 1,2,null
Store 1,3,499
Store 1,44,432
null,null,null
Store 2,1,355
Store 2,1,355
Store 2,null,345
Store 2,3,387
Store 2,4,312


In [0]:
display(df.filter(df.Store == 'Store 1').describe())

summary,Store,WeekInMonth,Revenue
count,4,4,3
mean,null,12.5,459.6666666666667
stddev,null,21.01586702153082,34.99047489436709
min,Store 1,1,432
max,Store 1,44,499


Esto dará el valor en un cuantil dado, en el intervalo de 0 a 1. Por lo tanto, si establece el segundo argumento en 0.0, obtendrá el valor más bajo para la columna. Con 1.0 obtienes el valor más alto. En el medio tienes la mediana, que es lo que se está buscando:

In [0]:
print(df.approxQuantile('Revenue', [0.5], 0))

[355.0]


## Pivot

A veces, desea cambiar sus datos de filas a columnas. La función se llama pivotar y está disponible en Pyspark.

Básicamente, estás rotando los datos alrededor de un eje determinado, de ahí el nombre.

En este caso, ese eje son los datos en una de sus columnas.

In [0]:
df_pivoted = df.groupBy('WeekInMonth').pivot('Store').sum('Revenue').orderBy('WeekInMonth')
display(df_pivoted)

WeekInMonth,null,Store 1,Store 2
null,null,null,345
1,null,448,710
2,null,null,null
3,null,499,387
4,null,null,312
44,null,432,null


In [0]:
display(df
.groupBy('Store','WeekInMonth')
.sum('Revenue')
.orderBy('WeekInMonth'))

Store,WeekInMonth,sum(Revenue)
null,null,null
Store 2,null,345
Store 2,1,710
Store 1,1,448
Store 1,2,null
Store 2,3,387
Store 1,3,499
Store 2,4,312
Store 1,44,432


In [0]:
display(df_pivoted.withColumnRenamed('Store 1','Store1')
        .withColumnRenamed('Store 2','Store2')
        .selectExpr('WeekInMonth',"stack(2, 'Store 1', Store1, 'Store 2', Store2) as (Store,Revenue)"))

WeekInMonth,Store,Revenue
null,Store 1,null
null,Store 2,345
1,Store 1,448
1,Store 2,710
2,Store 1,null
2,Store 2,null
3,Store 1,499
3,Store 2,387
4,Store 1,null
4,Store 2,312


## Explode

Hay otra situación con la que te encontrarás de vez en cuando. A veces llegan varios puntos de datos juntos en una columna. Esto usual cuando JSON es el formato de origen.

Puede resolver este problema utilizando el comando de Explode. Tomará la cadena con varios valores y los colocará en una fila cada uno.

In [0]:
from pyspark.sql.functions import explode
df = spark.createDataFrame([
(1, ['Rolex','Patek','Jaeger']),
(2, ['Omega','Heuer']),
(3, ['Swatch','Rolex'])],
('id','watches'))
display(df.withColumn('watches',explode(df.watches)))

id,watches
1,Rolex
1,Patek
1,Jaeger
2,Omega
2,Heuer
3,Swatch
3,Rolex


## Normalización

In [0]:
%fs ls /databricks-datasets/definitive-guide/data/simple-ml-scaling

path,name,size
dbfs:/databricks-datasets/definitive-guide/data/simple-ml-scaling/_SUCCESS,_SUCCESS,0
dbfs:/databricks-datasets/definitive-guide/data/simple-ml-scaling/part-00000-cd03406a-cc9b-42b0-9299-1e259fdd9382-c000.gz.parquet,part-00000-cd03406a-cc9b-42b0-9299-1e259fdd9382-c000.gz.parquet,1663


In [0]:
scaleDF = spark.read.parquet("/databricks-datasets/definitive-guide/data/simple-ml-scaling")
display(scaleDF)

id,features
0,"Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.1, -1.0))"
1,"Map(vectorType -> dense, length -> 3, values -> List(2.0, 1.1, 1.0))"
0,"Map(vectorType -> dense, length -> 3, values -> List(1.0, 0.1, -1.0))"
1,"Map(vectorType -> dense, length -> 3, values -> List(2.0, 1.1, 1.0))"
1,"Map(vectorType -> dense, length -> 3, values -> List(3.0, 10.1, 3.0))"


In [0]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MinMaxScaler_953bc9adb2cc__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  1|[3.0,10.1,3.0]|                 [10.0,10.0,10.0]|
+---+--------------+---------------------------------+



In [0]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+-----------------------------------+
| id|      features|StandardScaler_622c63d6ae39__output|
+---+--------------+-----------------------------------+
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  1|[3.0,10.1,3.0]|               [3.58568582800318...|
+---+--------------+-----------------------------------+

